columns: datetime, sentiment category, pols_mentioned, publication

In [102]:
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.models.widgets import DateRangeSlider,TextInput,Dropdown
from bokeh.models.widgets.inputs import AutocompleteInput
from bokeh.models import Panel, Tabs
from bokeh.layouts import column

import numpy as np

import datetime

In [3]:
pd.to_datetime(f'2016-07-{np.random.randint(0,30)} {np.random.randint(0,12)}:{np.random.randint(0,59)}:{np.random.randint(0,59)}')

Timestamp('2016-07-29 09:39:25')

In [4]:
pollist = ['Donald Trump', 'Mike Pence', 'Mike Pompeo', 'Steven Mnuchin',
       'Mark T. Esper', 'William Barr', 'David Bernhardt', 'Sonny Perdue',
       'Wilbur Ross', 'Patrick Pizzella',
       'Carol Miller', 'Bryan Steil', 'Mark Pocan', 'Ron Kind', 'Gwen Moore',
       'Jim Sensenbrenner', 'Glenn Grothman', 'Sean Duffy', 'Mike Gallagher',
       'Liz Cheney']

In [5]:
publist = ['Refinery29.com',
'Motorauthority.com',
'Sbnation.com',
'Hrbartender.com',
'Cincinnati.com',
'Pjmedia.com',
'Mediaite.com',
'The Jerusalem Post',
'Relevantmagazine.com',
'Theregister.co.uk',
'Fark.com',
'Autoblog.com',
'Tor.com',
'Chicagotribune.com',
'Memeorandum.com',
'Denofgeek.com',
'MTV News',
'Businessoffashion.com']

In [6]:
fake_list=[]
for i in range(10_000):
    fake_list.append(
    {'date':pd.to_datetime(f'2016-07-{np.random.randint(1,30)} {np.random.randint(1,12)}:{np.random.randint(0,59)}:{np.random.randint(0,59)}'),
    'pol':np.random.choice(pollist),
     'pub':np.random.choice(publist),
     'sentiment':np.random.randint(0,2)
    }
    )

In [8]:
fake_df=pd.DataFrame(fake_list)

In [18]:
fake_df['date_only']=fake_df['date'].apply(lambda x: x.date())

In [35]:
dailypos = fake_df[fake_df['sentiment']==0].groupby('date_only').count()['sentiment']

In [119]:
output_file("line.html")

pnet = figure(plot_width=1000, plot_height=400, x_axis_type='datetime')
pnet.toolbar.logo = None
pnet.toolbar_location = None
pnet.background_fill_color = "lavender"

pvol = figure(plot_width=1000, plot_height=100, x_axis_type='datetime')
pvol.toolbar.logo = None
pvol.toolbar_location = None
pvol.background_fill_color = "bisque"

# add a line renderer
pnet.line(dailycounts['date'], 
       dailycounts['sentiment_net'], 
       line_width=2,
      color='purple')

pvol.line(dailycounts['date'], 
       dailycounts['volume'], 
       line_width=2,color='orange')



# pubmenu = [(x,x) for x in publist]
# polmenu = [(x,x) for x in pollist]
# pubdropdown = Dropdown(label="Publications", button_type="warning", menu=pubmenu)
# poldropdown = Dropdown(label="Politicians", button_type="warning", menu=polmenu)
polmenu = AutocompleteInput(completions=pollist,title='Politician (case sensitive)')
pubmenu = AutocompleteInput(completions=publist,title='Publication (case sensitive)')
date_slider = DateRangeSlider(start=min(dailycounts['date']), end=max(dailycounts['date']), 
                          value=(min(dailycounts['date']),max(dailycounts['date'])), step=1, title="Date Range")
show(column(pnet,pvol,date_slider,pubmenu,polmenu))

In [65]:
dailycounts =pd.DataFrame(fake_df[fake_df['sentiment']==0]['date_only'].unique(),columns=['date']).sort_values(by='date')

In [78]:
dailycounts['sentiment0']=fake_df[fake_df['sentiment']==0].groupby('date_only').count()['sentiment'].values

In [79]:
dailycounts['sentiment1']=fake_df[fake_df['sentiment']==1].groupby('date_only')['sentiment'].count().values

In [114]:
dailycounts['sentiment_net'] = (fake_df[fake_df['sentiment']==0].groupby('date_only').count()['sentiment'].values 
                                - fake_df[fake_df['sentiment']==1].groupby('date_only')['sentiment'].count().values)/ fake_df.groupby('date_only')['sentiment'].count().values

In [115]:
dailycounts['volume'] = fake_df.groupby('date_only')['sentiment'].count().values

In [116]:
dailycounts.head()

,date,sentiment0,sentiment1,sentiment_net,volume
1,2016-07-01,181,155,0.077381,336
27,2016-07-02,195,166,0.080332,361
23,2016-07-03,170,153,0.052632,323
9,2016-07-04,185,162,0.066282,347
2,2016-07-05,188,180,0.021739,368


In [70]:
type(fake_df[fake_df['sentiment']==0].groupby('date_only').count()['sentiment'])


pandas.core.series.Series

In [74]:
fake_df[fake_df['sentiment']==1].groupby('date_only')['sentiment'].count().shape

(29,)

(29, 3)